## Final Project Submission

Please fill out:
* Student name: Gregory Antony Mikuro
* Student pace: Full Time
* Scheduled project review date/time: 22/03/2024
* Instructor name: Nikita Njoroge
* Blog post URL: 


In [ ]:
## Your code here - remember to use markdown cells for comments as well!

## Data Preparation

### 1. Box Office Mojo 

### 2. IMDB

### 3. TheMovieDB

### 4. The Numbers